# Neighbour Extraction using Gram Matrices

---

In [ ]:
import sys
print(sys.executable)
print(sys.version)

In [ ]:
from ipynb.fs.full.Helper import getDataLoader
from ipynb.fs.full.GramMatrix import convertModel, GramMatrixLayer

import torch
import torch.nn as nn

%matplotlib inline

In [ ]:
loader = getDataLoader('/scratch/bam_subset_2_0', batch_size=4, shuffle=False, num_workers=4, testing=True)

In [ ]:
import torchvision.models as models
vgg19 = models.vgg19(pretrained=True)

In [ ]:
gramMatrixLayers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']
vgg19, model, gram_matrices = convertModel(vgg19, gramMatrixLayers, testing=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model.eval()
model.to(device)

In [ ]:
import numpy as np

RPM = np.random.normal(0, 1/1000, (610304, 1000))

In [ ]:
RPM = torch.from_numpy(RPM)
RPM = RPM.cuda()
RPM = RPM.float()
print(RPM.shape)

torch.save(RPM, "/scratch/kshitij98/rpm")

In [ ]:
import time

dataIter = iter(loader)
t = time.time()
X = []
for i in range(100):
    (data, classes), names = dataIter.next()
    data = data.cuda()
    out = model(data)
    G = []
    for layer in gram_matrices:
        G.append(layer.gramMatrix)
    G = torch.cat(G, 1)
    G = torch.mm(G, RPM)
    for i, gm in enumerate(G):
        torch.save(gm, names[i].replace('bam_subset_2_0', 'bam_subset_2_0_features')
#     X.append(G)
    print(i+1, "\tETA: ", ((time.time() - t) / ((i + 1) * 4)) * (121000 - ((i+1) * 4)) * (1 / 60), "minutes", end='\r')
# X = torch.cat(X, 0)

print("\n")

print(X.shape)
    

---

## Get Neighbours